# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,9.12,74,75,3.60,CA,1696110670
1,1,waitangi,-43.9535,-176.5597,11.01,77,89,4.02,NZ,1696110822
2,2,port-aux-francais,-49.3500,70.2167,4.54,97,100,12.98,TF,1696110823
3,3,albany,42.6001,-73.9662,19.21,87,79,0.45,US,1696110621
4,4,college,64.8569,-147.8028,9.00,51,75,5.14,US,1696110823


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    alpha = 0.3
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_narrowed_df= city_data_df.loc[(city_data_df["Max Temp"]< 28)
                                        & (22< city_data_df["Max Temp"])
                                        & (city_data_df["Wind Speed"] < 5)
                                        & (city_data_df["Cloudiness"] < 30)
                                        & (city_data_df["Humidity"] <=60)
                                        ]

# Drop any rows with null values
city_data_dropna_df = city_data_narrowed_df.dropna()

# Display sample data
city_data_dropna_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,lata,40.1629,-8.3327,23.84,55,5,0.62,PT,1696110830
81,81,macia,-25.0269,33.0989,25.20,45,0,3.67,MZ,1696110857
172,172,sidney,-33.8679,151.2073,25.75,51,0,4.12,AU,1696110700
174,174,ghanzi,-21.5667,21.7833,23.04,14,0,3.45,BW,1696110897
212,212,maler kotla,30.5167,75.8833,22.13,45,0,2.54,IN,1696110913
319,319,alice springs,-23.7000,133.8833,23.75,20,3,4.12,AU,1696110959
351,351,north bondi,-33.8824,151.2769,26.63,51,0,4.12,AU,1696110973
357,357,cuamba,-14.8031,36.5372,24.69,29,28,1.77,MZ,1696110976
407,407,khangarh,29.9173,71.1618,24.04,27,0,1.26,PK,1696110996
432,432,port macquarie,-31.4333,152.9167,22.16,55,2,3.96,AU,1696110905


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_dropna_df.loc[:, ["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
18,lata,PT,40.1629,-8.3327,55,
81,macia,MZ,-25.0269,33.0989,45,
172,sidney,AU,-33.8679,151.2073,51,
174,ghanzi,BW,-21.5667,21.7833,14,
212,maler kotla,IN,30.5167,75.8833,45,
319,alice springs,AU,-23.7000,133.8833,20,
351,north bondi,AU,-33.8824,151.2769,51,
357,cuamba,MZ,-14.8031,36.5372,29,
407,khangarh,PK,29.9173,71.1618,27,
432,port macquarie,AU,-31.4333,152.9167,55,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
hotel_df.loc[18, "Lat"]

40.1629

In [20]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "limit":100,
    "apiKey": geoapify_key,
    "format": "json",
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lata - nearest hotel: Residencial Botânico
macia - nearest hotel: No hotel found
sidney - nearest hotel: The Westin
ghanzi - nearest hotel: Ghanzi farmhouse
maler kotla - nearest hotel: No hotel found
alice springs - nearest hotel: Aurora Alice Springs
north bondi - nearest hotel: Beach Road Hotel
cuamba - nearest hotel: Pensao Cariaco
khangarh - nearest hotel: No hotel found
port macquarie - nearest hotel: The Observatory
karachi - nearest hotel: Quetta Agha Hotel
behbahan - nearest hotel: هتل بین المللی رضا
az zubayr - nearest hotel: فندق برج بابل
gursahaiganj - nearest hotel: No hotel found
karratha - nearest hotel: Karratha International Hotel
johi - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
18,lata,PT,40.1629,-8.3327,55,Residencial Botânico
81,macia,MZ,-25.0269,33.0989,45,No hotel found
172,sidney,AU,-33.8679,151.2073,51,The Westin
174,ghanzi,BW,-21.5667,21.7833,14,Ghanzi farmhouse
212,maler kotla,IN,30.5167,75.8833,45,No hotel found
319,alice springs,AU,-23.7000,133.8833,20,Aurora Alice Springs
351,north bondi,AU,-33.8824,151.2769,51,Beach Road Hotel
357,cuamba,MZ,-14.8031,36.5372,29,Pensao Cariaco
407,khangarh,PK,29.9173,71.1618,27,No hotel found
432,port macquarie,AU,-31.4333,152.9167,55,The Observatory


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 800,
    size = "Humidity",
    color = "City",
    alpha = 0.8,
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)